# Customer Churn Prediction

In [86]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical

## Importing the dataset

In [2]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
df.shape

(10000, 14)

In [5]:
df.duplicated().sum()

0

In [6]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

<br>This basic EDA clearly shows that there is no duplicate entries or null entries in the dataset. <br>
So good to go!

## Attributes Analysis

In [7]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [8]:
df["RowNumber"] # to be removed

0           1
1           2
2           3
3           4
4           5
        ...  
9995     9996
9996     9997
9997     9998
9998     9999
9999    10000
Name: RowNumber, Length: 10000, dtype: int64

In [9]:
len(df['CustomerId'].unique()) == len(df)  # unique for everyone - no insights can be obtained - to be removed

True

In [10]:
len(df["Surname"].unique())  # Name usually don't contribute to churn - but surname can also mean same family -> exit of one from family can lead to removal of others also

2932

In [11]:
len(df["CreditScore"].unique()) # might contribute

460

In [12]:
df["Geography"].unique()  # Contributing factor

array(['France', 'Spain', 'Germany'], dtype=object)

In [13]:
df['Gender'].unique() # Contributing

array(['Female', 'Male'], dtype=object)

In [14]:
df['Age'].unique() # contributing

array([42, 41, 39, 43, 44, 50, 29, 27, 31, 24, 34, 25, 35, 45, 58, 32, 38,
       46, 36, 33, 40, 51, 61, 49, 37, 19, 66, 56, 26, 21, 55, 75, 22, 30,
       28, 65, 48, 52, 57, 73, 47, 54, 72, 20, 67, 79, 62, 53, 80, 59, 68,
       23, 60, 70, 63, 64, 18, 82, 69, 74, 71, 76, 77, 88, 85, 84, 78, 81,
       92, 83], dtype=int64)

In [15]:
df['Tenure'].unique() # tenure is contributing factor

array([ 2,  1,  8,  7,  4,  6,  3, 10,  5,  9,  0], dtype=int64)

In [16]:
df['Balance'].min(), df['Balance'].max() # contributing

(0.0, 250898.09)

In [17]:
df['NumOfProducts'].unique() # contributing

array([1, 3, 2, 4], dtype=int64)

In [18]:
df['HasCrCard'].unique() # yes might contribute

array([1, 0], dtype=int64)

In [19]:
df['IsActiveMember'].unique() # contributing

array([1, 0], dtype=int64)

In [20]:
df['EstimatedSalary'].unique() # might contribute

array([101348.88, 112542.58, 113931.57, ...,  42085.58,  92888.52,
        38190.78])

In [21]:
df['Exited'].unique() # target variable -  1 means exit; 0 means retain

array([1, 0], dtype=int64)

In [22]:
# Dropping RowNumber, CustomerId
df.drop(['RowNumber', 'CustomerId'], axis=1, inplace=True)

In [23]:
df.head()

,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [24]:
# Label Encoding the categorical variables
attr = ['Surname', 'Geography', 'Gender']
for col in attr:
    encoder = LabelEncoder()
    df[col] = encoder.fit_transform(df[col])
df.head()

,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1115,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,1177,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,2040,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,289,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,1822,850,2,0,43,2,125510.82,1,1,1,79084.10,0


## Train Test Split

In [29]:
X = df.drop('Exited', axis=1)
y = df['Exited']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Logistic Regression

In [33]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7995

In [35]:
kfold_lr = LogisticRegression()
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(kfold_lr, X, y, cv=kfold)
cv_scores

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

array([0.7995, 0.787 , 0.7805, 0.7955, 0.792 ])

In [36]:
# KFold validation for Logistic Regression
cv_scores.mean()

0.7909

## Decision Tree

In [38]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test, y_pred)

0.7855

## Random Forest

In [40]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8645

## GradBoost

In [43]:
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)
accuracy_score(y_test, y_pred)

0.8645

## SVM

In [45]:
svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy_score(y_test, y_pred)

0.8035

## KNN

In [47]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_pred, y_test)

0.7645

## Neural Network

In [82]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, to_categorical(y_train, num_classes=2), epochs=100)

Epoch 1/100
250/250 [==============================] - 4s 4ms/step - loss: 108.5715 - accuracy: 0.6675
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 32.3481 - accuracy: 0.6869
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 18.3378 - accuracy: 0.6762
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 12.3200 - accuracy: 0.6779
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 4.5110 - accuracy: 0.6860
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 1.5957 - accuracy: 0.6988
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 1.4275 - accuracy: 0.7034
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 1.4652 - accuracy: 0.6949
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 1.2924 - accuracy: 0.7057
Epoch 10/100
250/250 [==============================] - 1s 5ms/step - loss: 0.9378 - a

In [83]:
loss, acc = model.evaluate(X_test, to_categorical(y_test, num_classes=2))
print("Loss:", loss)
print("Accuracy:", acc)

63/63 [==============================] - 1s 3ms/step - loss: 0.4959 - accuracy: 0.8035
Loss: 0.4958919286727905
Accuracy: 0.8034999966621399


## Exploring: Parameter tuning for Random Forest

In [87]:
# Example with Random Forest Classifier
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


In [84]:
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Evaluate on test set
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters found:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
              precision    recall  f1-score   support

           0       0.88      0.97      0.92      1607
           1       0.76      0.45      0.57       393

    accuracy                           0.86      2000
   macro avg       0.82      0.71      0.74      2000
weighted avg       0.86      0.86      0.85      2000



<br>From this we can conclude that Random Forest and GradBoost are classifying the Customer Churn better. <br>
Hence, a model to classify Customer Churn was successfully built with an accuracy of 86%